In [37]:
import nfl_data_py as nfl
import pandas as pd
from NFL_Player import Player
from Injury import *
import requests
from bs4 import BeautifulSoup
from helperFunctions import *
from googlesearch import search
import re
from datetime import datetime
import concurrent.futures
from tqdm import tqdm
import time
import random

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1'
]

In [2]:
nflInjuries20 = nfl.import_injuries([2020])
nflInjuries21 = nfl.import_injuries([2021])
nflInjuries22 = nfl.import_injuries([2022])
nflInjuries23 = nfl.import_injuries([2023])
nflInjuries24 = nfl.import_injuries([2024])
allNFLInjuries = pd.concat([nflInjuries20, nflInjuries21, nflInjuries22, nflInjuries23, nflInjuries24])

max_season = max(allNFLInjuries["season"])
max_week = max(allNFLInjuries.loc[(allNFLInjuries["season"] == max_season)]["week"])

historicalNFLInjuries = allNFLInjuries.loc[(allNFLInjuries["season"] != max_season) | (allNFLInjuries["week"] != max_week)].copy()
historicalNFLInjuries['InjuredInGame?'] = historicalNFLInjuries['practice_status'].apply(lambda x: x.strip() == "")

In [38]:
def players_and_injuries_conversion_nfl(historicalNFLInjuries):
    players = {}
    for index, row in historicalNFLInjuries.iterrows():
        keysList = list(players.keys())
        temp_injury = GameInjuryReport(
            row["season"], 
            row["game_type"], 
            row["team"], 
            row["week"], 
            row["report_primary_injury"], 
            row["report_secondary_injury"], 
            row["report_status"], 
            row["practice_primary_injury"], 
            row["practice_secondary_injury"], 
            row["practice_status"], 
            row["InjuredInGame?"]
        )

        if row["full_name"] not in keysList:
            players[row["full_name"]] = Player(row["first_name"], row["last_name"], row["position"])
        players.get(row["full_name"]).add_injury(temp_injury)
    return players

def get_draft_sharks_url(first_name, last_name):
    # Clean up names for URL generation
    first_name_clean = ''.join(char for char in first_name if char.isalnum()).lower()
    last_name_clean = ''.join(char for char in last_name if char.isalnum()).lower()
    full_name = f"{first_name} {last_name}"
    
    # Prepare search query
    search_query = f"{full_name} Draft Sharks"

    # Maximum retries if rate-limited
    max_retries = 5
    retries = 0

    # Start searching
    while retries < max_retries:
        try:
            # Rotate user-agent for each request
            headers = {'User-Agent': random.choice(user_agents)}
            
            # Adding random sleep between 3 and 6 seconds before requests
            random_sleep = random.uniform(3, 6)
            print(f"Waiting {random_sleep:.2f} seconds before making the search request...")
            time.sleep(random_sleep)

            # Perform the search and look for the Draft Sharks URL
            for url in search(search_query, num_results=10):
                if 'draftsharks.com' in url:
                    if f"{first_name_clean}-{last_name_clean}" in url:
                        # Return the injury history URL if found
                        if 'injury-history' in url:
                            return url

                        # If no injury history found, construct it using player ID
                        match = re.search(r'/(\d+)$', url)
                        if match:
                            player_id = match.group(1)
                            return f"https://www.draftsharks.com/fantasy/injury-history/{first_name_clean}-{last_name_clean}/{player_id}"
            break  # Exit if no URL found in this search cycle

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:  # Handle rate limit errors
                wait_time = 2 ** retries + random.uniform(1, 3)  # Exponential backoff
                print(f"Rate limited. Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
                retries += 1  # Increment retry count
            else:
                print(f"HTTP Error: {e}")
                return None

        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    return None

def players_and_injuries_conversion_draftsharks(player_name, player):
    url = get_draft_sharks_url(player.first_name, player.last_name)
    full_name = player_name
    if url:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
        except requests.exceptions.HTTPError as err:
            print(f"HTTP error for {full_name}: {err}")
            retry_after = response.headers.get("Retry-After")
            if retry_after:
                print("Stopped for: " + retry_after)
                time.sleep(int(retry_after))
                return players_and_injuries_conversion_draftsharks(player_name, player)  # Retry after waiting

        except Exception as e:
            print(f"Error fetching data for {full_name}: {e}")
            return

        soup = BeautifulSoup(response.content, 'html.parser')
        table_rows = soup.find("div", class_="injury-history-table-container").find("tbody").findAll("tr")

        for row in table_rows:
            row_data = row.findAll("td")
            injury_date = datetime.strptime(row_data[0].text, '%b %d, %Y')  # Date
            injury_league = row_data[1].text  # League
            injury = row_data[2].text  # Injury
            injury_details = row_data[3].text  # Description
            temp_injury = DetailedInjuryReport(injury_date, injury, injury_league, injury_details)
            player.add_injury(temp_injury)

        print(f"Draft Sharks URL for {full_name}: {url}")
    else:
        print(f"No Draft Sharks URL found for {full_name}.")
def split_dictionary(original_dict, chunk_size=450):
    # Split the original dictionary into smaller chunks
    keys = list(original_dict.keys())
    chunks = [dict((k, original_dict[k]) for k in keys[i:i + chunk_size]) 
              for i in range(0, len(keys), chunk_size)]
    return chunks

In [4]:
# x = filterHistoricalNFLInjuries(historicalNFLInjuries, position="WR")
playersWithInjuries = players_and_injuries_conversion_nfl(historicalNFLInjuries)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(players_and_injuries_conversion_draftsharks, player_name, player): player_name for player_name, player in playersWithInjuries.items()}
    for future in concurrent.futures.as_completed(futures):
        player_name = futures[future]
        try:
            future.result()  # This will raise an exception if the thread raised one
        except Exception as e:
            print(f"Exception occurred for player {player_name}: {e}")


No Draft Sharks URL found for Alex Mack.
No Draft Sharks URL found for James Carpenter.
No Draft Sharks URL found for Matt Hennessy.
Draft Sharks URL for DeAndre Hopkins: https://www.draftsharks.com/fantasy/injury-history/deandre-hopkins/6679
Draft Sharks URL for Russell Gage: https://www.draftsharks.com/fantasy/injury-history/russell-gage/10070
Draft Sharks URL for Maxx Williams: https://www.draftsharks.com/fantasy/injury-history/maxx-williams/7240
Draft Sharks URL for Larry Fitzgerald: https://www.draftsharks.com/fantasy/injury-history/larry-fitzgerald/1306
HTTP error for Josh Jones: 404 Client Error: Not Found for url: https://www.draftsharks.com/fantasy/injury-history/josh-jones/9936
Exception occurred for player Josh Jones: 'NoneType' object has no attribute 'find'Draft Sharks URL for Todd Gurley: https://www.draftsharks.com/fantasy/injury-history/todd-gurley/7208
Draft Sharks URL for Dante Fowler: https://www.draftsharks.com/fantasy/injury-history/dante-fowler/7222

Draft Sharks 

In [40]:
again = []

# Wrap the playersWithInjuries dictionary with tqdm for progress tracking
for player in tqdm(playersWithInjuries, desc="Processing Players"):
    if not any(isinstance(item, DetailedInjuryReport) for item in playersWithInjuries[player].injuries):
        url = get_draft_sharks_url(playersWithInjuries[player].first_name, playersWithInjuries[player].last_name)
        if url:
            again.append(player)
            print(f"Draft Sharks URL for {player}: {url}")
        else:
            print(f"NO Draft Sharks URL for {player}")
print(len(again))

Processing Players:   0%|          | 0/2772 [00:00<?, ?it/s]

Waiting 3.63 seconds before making the search request...
Rate limited. Retrying in 3.00 seconds...
Waiting 4.44 seconds before making the search request...
Rate limited. Retrying in 3.41 seconds...
Waiting 5.88 seconds before making the search request...
Rate limited. Retrying in 6.97 seconds...
Waiting 5.47 seconds before making the search request...
Rate limited. Retrying in 9.07 seconds...
Waiting 4.36 seconds before making the search request...
Rate limited. Retrying in 18.72 seconds...


Processing Players:   0%|          | 1/2772 [01:07<52:01:54, 67.60s/it]

NO Draft Sharks URL for KeeSean Johnson
Waiting 5.69 seconds before making the search request...
Rate limited. Retrying in 2.18 seconds...
Waiting 5.36 seconds before making the search request...
Rate limited. Retrying in 4.42 seconds...
Waiting 4.63 seconds before making the search request...
Rate limited. Retrying in 6.81 seconds...
Waiting 3.83 seconds before making the search request...
Rate limited. Retrying in 9.62 seconds...
Waiting 4.98 seconds before making the search request...
Rate limited. Retrying in 17.47 seconds...


Processing Players:   0%|          | 5/2772 [02:15<18:50:00, 24.50s/it]

...

NO Draft Sharks URL for Troy Pride
Waiting 5.28 seconds before making the search request...
Rate limited. Retrying in 3.29 seconds...
Waiting 4.42 seconds before making the search request...
Rate limited. Retrying in 3.32 seconds...
Waiting 3.90 seconds before making the search request...
Rate limited. Retrying in 5.69 seconds...
Waiting 4.77 seconds before making the search request...
Rate limited. Retrying in 9.96 seconds...
Waiting 4.97 seconds before making the search request...
Rate limited. Retrying in 18.68 seconds...


Processing Players:   1%|▏         | 41/2772 [25:49<37:04:07, 48.86s/it]

NO Draft Sharks URL for Trenton Scott
Waiting 3.72 seconds before making the search request...
Rate limited. Retrying in 3.72 seconds...
Waiting 4.88 seconds before making the search request...
Rate limited. Retrying in 4.88 seconds...
Waiting 3.26 seconds before making the search request...
Rate limited. Retrying in 6.51 seconds...
Waiting 5.96 seconds before making the search request...
Rate limited. Retrying in 9.69 seconds...
Waiting 4.11 seconds before making the search request...
Rate limited. Retrying in 17.19 seconds...


Processing Players:   2%|▏         | 46/2772 [26:55<20:41:08, 27.32s/it]

NO Draft Sharks URL for Brandon Zylstra
Waiting 3.27 seconds before making the search request...
